In [2]:
import numpy as np
import pandas as pd 
import yfinance as yf # Yahoo finance 에서 제공하는 데이터에 접근 가능
import matplotlib as plt
import seaborn as sns

In [3]:
df_price = pd.read_csv('01-삼성전자-주가.csv', encoding='utf8')
df_price.describe()

,일자,시가,고가,저가,종가,거래량
count,9.288000e+03,9288.000000,9288.000000,9288.000000,9288.000000,9.288000e+03
mean,2.001347e+07,11108.556417,11228.754522,10986.106481,11108.235465,1.645823e+07
std,1.015376e+05,13792.646300,13920.105135,13658.809496,13790.922497,1.701535e+07
min,1.985010e+07,116.000000,116.000000,115.000000,116.000000,0.000000e+00
25%,1.992121e+07,624.000000,632.000000,620.000000,623.000000,3.833986e+06
50%,2.001032e+07,5045.000000,5190.000000,4955.000000,5075.000000,1.199608e+07
75%,2.010080e+07,15920.000000,16050.000000,15740.000000,15920.000000,2.284080e+07
max,2.020011e+07,57500.000000,57520.000000,56760.000000,57220.000000,3.266220e+08


In [4]:
pd.to_datetime(df_price['일자'], format='%Y%m%d')

df_price['일자'] = pd.to_datetime(df_price['일자'], format='%Y%m%d')
df_price['연도'] =df_price['일자'].dt.year
df_price['월'] =df_price['일자'].dt.month
df_price['일'] =df_price['일자'].dt.day

In [5]:
plt.figure(figsize=(16, 9))
sns.lineplot(y=df['종가'], x=df['일자'])
plt.xlabel('time')
plt.ylabel('price')

TypeError: 'module' object is not callable

In [6]:
df = df_price

In [7]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scale_cols = ['시가', '고가', '저가', '종가', '거래량']
df_scaled = scaler.fit_transform(df[scale_cols])

df_scaled = pd.DataFrame(df_scaled)
df_scaled.columns = scale_cols

print(df_scaled)

            시가        고가        저가        종가       거래량
0     0.968632  0.980489  0.979522  0.975133  0.030291
1     0.954691  0.966553  0.961868  0.969880  0.031470
2     0.973860  0.983973  0.967164  0.969880  0.047217
3     0.965147  0.973521  0.968929  0.964626  0.039781
4     0.977346  0.983973  0.981287  0.975133  0.025585
...        ...       ...       ...       ...       ...
9283  0.000174  0.000174  0.000124  0.000123  0.000995
9284  0.000227  0.000226  0.000212  0.000193  0.002587
9285  0.000227  0.000244  0.000229  0.000228  0.002363
9286  0.000227  0.000226  0.000229  0.000210  0.000332
9287  0.000244  0.000244  0.000247  0.000228  0.000342

[9288 rows x 5 columns]


In [8]:
window_size = 20
#TEST_SIZE = 200

In [9]:
train = df_scaled
#test = df_scaled[-TEST_SIZE:]

In [10]:
def make_dataset(data, label, window_size):
    feature_list = []
    label_list = []
    for i in range(len(data) - window_size):
        feature_list.append(np.array(data.iloc[i:i+window_size]))
        label_list.append(np.array(label.iloc[i+window_size]))
    return np.array(feature_list), np.array(label_list)

In [11]:
feature_cols = ['시가', '고가', '저가', '거래량']
label_cols = ['종가']

train_feature = train[feature_cols]
train_label = train[label_cols]


In [12]:

# train dataset
train_feature, train_label = make_dataset(train_feature, train_label, 20)

# train, validation set 생성
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(train_feature, train_label, test_size=0.2)

x_train.shape, x_valid.shape
# ((6086, 20, 4), (1522, 20, 4))

# test dataset (실제 예측 해볼 데이터)
#test_feature = test[feature_cols]
#test_feature.reset_index(drop=True, inplace=True)
#test_label = test[label_cols]
#test_label.reset_index(drop=True, inplace=True)
#test_feature, test_label = make_dataset(test_feature, test_label, 20)
#test_feature.shape, test_label.shape
# ((180, 20, 4), (180, 1))


((7414, 20, 4), (1854, 20, 4))

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Bidirectional 
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model = Sequential()
model.add(Bidirectional(LSTM(16, 
               input_shape=(train_feature.shape[1], train_feature.shape[2]), 
               activation='relu',
               return_sequences=False))
          )
model.add(Dense(1))

C:\Users\loven\Anaconda3\envs\allocation\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\loven\Anaconda3\envs\allocation\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\loven\Anaconda3\envs\allocation\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\loven\Anaconda3\envs\allocation\lib\site-packages

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [14]:
model.compile(loss='mean_squared_error', optimizer='adam')
early_stop = EarlyStopping(monitor='val_loss', patience=5)
filename = os.path.join('tmp_checkpoint.h5')
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

history = model.fit(x_train, y_train, 
                    epochs=200, 
                    batch_size=16,
                    validation_data=(x_valid, y_valid), 
                    callbacks=[early_stop, checkpoint])

Train on 7414 samples, validate on 1854 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/200
7344/7414 [============================>.] - ETA: 0s - loss: 2.9295 - acc: 0.0011   
Epoch 00001: val_loss improved from inf to 2.98775, saving model to tmp_checkpoint.h5
7414/7414 [==============================] - 8s 1ms/sample - loss: 2.9329 - acc: 0.0011 - val_loss: 2.9877 - val_acc: 0.0022
Epoch 2/200
7408/7414 [============================>.] - ETA: 0s - loss: 2.9323 - acc: 0.0011 
Epoch 00002: val_loss did not improve from 2.98775
7414/7414 [==============================] - 6s 833us/sample - loss: 2.9329 - acc: 0.0011 - val_loss: 2.9877 - val_acc: 0.0022
Epoch 3/200
7376/7414 [============================>.] - ETA: 0s - loss: 2.9364 - acc: 0.0011  
Epoch 00003: val_loss did not improve from 2.98775
7414/7414 [==============================] - 6s 838us/sample - loss: 2.9329 - acc: 0.0011 - val_loss: 2.9877 - val_acc: 0.0022
Epo

In [1]:
# weight 로딩
model.load_weights(filename)

# 예측
pred = model.predict(test_feature)

NameError: name 'model' is not defined

In [ ]:
plt.figure(figsize=(12, 9))
plt.plot(test_label, label='actual')
plt.plot(pred, label='prediction')
plt.legend()
plt.show()